# 1. Importation des librairies

In [0]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
from keras.preprocessing import image
from keras.models import Model, load_model
import numpy as np
import os
import cv2 
import os
import csv
import tensorflow as tf
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.xception import Xception, preprocess_input, decode_predictions #299*299
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions #224*224
from keras.applications.inception_v3 import InceptionV3, preprocess_input,decode_predictions# input shape= 299x299
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input,decode_predictions# input shape= 299x299
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions# input shape= 224x224 
from keras.applications.densenet import DenseNet169, preprocess_input
from keras.applications.densenet import DenseNet201, preprocess_input
from keras.applications.nasnet import NASNetLarge, preprocess_input
from keras.applications.nasnet import NASNetMobile, preprocess_input
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance

# 2. Téléchargement de la base de données (commençons par la petite : Small)

In [0]:
! wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/defi1certif-datasets-fire_small.tar

# 3. Décompression du dossier vers un le nouveau dossier "bases/small"

In [0]:
bases_path_after="bases"
# Création du dossier pour sauvegrader le model
if os.path.exists(bases_path_after) == False:
    os.makedirs(bases_path_after)

In [0]:
!tar xf defi1certif-datasets-fire_small.tar -C 'bases' --one-top-level && mv bases/defi1certif-datasets-fire_small bases/small

# 4. Création du fichier "classes.txt" (labels des données d'entrainement): 

In [0]:
!for d in bases/small/*;do [[ -d "$d" ]] && echo "${d##bases/small/}" >> classes.txt; done

# 5. Génération des données à partir de chemins

In [0]:
def generate_from_paths_and_labels(input_paths, labels, batch_size, input_size=(299,299)):

    num_samples = len(input_paths)
    while 1:
        perm = np.random.permutation(num_samples)
        input_paths = input_paths[perm]
        labels = labels[perm]
        for i in range(0, num_samples, batch_size):
            inputs = list(map(
                lambda x: image.load_img(x, target_size=input_size),
                input_paths[i:i+batch_size]
            ))
            inputs = np.array(list(map(
                lambda x: image.img_to_array(x),
                inputs
            )))
            inputs = preprocess_input(inputs)
            yield (inputs, labels[i:i+batch_size])

# 6. Définition des paramètres (l'interface à droite vous permet de changer les paramètres aussi)

In [0]:
nb_classes = 3
nbr_batch_size=8 #@param [1,2,4,8,16,32,64,128] {type:"raw"}
dataset_path = "bases"
dataset_name='small' #@param ["small","medium","big","personal"]

dataset_path = os.path.join('bases/', dataset_name)
classes_path = "classes.txt"
csv_path = 'result.csv'
epochs = 10 #@param {type:"slider", min:5, max:100, step:5}

seed = 1
classifier = "VGG16" #@param ["SqueezeNet","Xception","VGG16","VGG19","ResNet50","InceptionV3","InceptionResNetV2","MobileNet","DenseNet121","DenseNet169","DenseNet201","NASNetLarge","NASNetMobile"] {type:"string"}
result_path = 'results/'+classifier
log={
    'epochs':epochs,
    'batch_size':nbr_batch_size,
    'val_loss':-1,
    'val_acc':-1,
}

# 7. Récupérer les images ainsi que leurs classes  (étape pouvant prendre quelques minutes)

In [0]:
# Récupurer les noms des classes
with open(classes_path, 'r') as f:
    classes = f.readlines()
    classes = list(map(lambda x: x.strip(), classes))
num_classes = len(classes)

# Récupurer les images et les classes
input_paths, labels = [], []
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    class_id = classes.index(class_name)
    for path in os.listdir(class_path):
        path = os.path.join(class_path, path)
        if imghdr.what(path) == None:
            # this is not an image file
            continue
        input_paths.append(path)
        labels.append(class_id)

# 8. Préparer les données

In [0]:
# Question 1: Convertir les lebels vers le format one-hot-vector
???????????????????????????????????????

# Question 2: Convertir "convert "input paths" vers le format numpy
???????????????????????????????????????

# shuffle dataset (permuter les données)
perm = np.random.permutation(len(input_paths))
labels = labels[perm]
input_paths = input_paths[perm]

# Question3 : Diviser les données en deux parties : entrainement et validation
???????????????????????????????????????

train_labels, val_labels = labels[:border], labels[border:]
train_input_paths, val_input_paths = input_paths[:border], input_paths[border:]
print("Training on %d images and labels" % (len(train_input_paths)))
print("Validation on %d images and labels" % (len(val_input_paths)))

# Création du dossier pour sauvegrader le model
if os.path.exists(result_path) == False:
    os.makedirs(result_path)

# 9. Téléchargement d'un modèle pré-entraîné et Transfer Learning

In [0]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))  # La pouvez tester différentes architectures

# create a custom top classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.inputs, outputs=predictions)

# 9. Entraînement du modèle 

In [0]:

# ====================================================
# Train the whole model
# ====================================================
# set all the layers to be trainable
for layer in model.layers:
    layer.trainable = True

# Question 4: compiler le modèle "model"
????????????????????????????????????????????????????????

# Création du dossier pour sauvegrader le model
if os.path.exists(result_path) == False:
    os.makedirs(result_path)

# Question 5 : Compléter l'appel a fonction fit_generator pour l'entrainement 
history=model.fit_generator(
    generator=generate_from_paths_and_labels(
        input_paths=train_input_paths,
        labels=train_labels,
        batch_size=nbr_batch_size,
        input_size=(???,???,3)
        
    ),
    steps_per_epoch=math.ceil(len(train_input_paths) / nbr_batch_size),
    epochs=epochs,
    validation_data=generate_from_paths_and_labels(
        input_paths=???,
        labels=???,
        batch_size=nbr_batch_size,
        input_size=(???,???,3)
    ),
    validation_steps=math.ceil(len(val_input_paths) / nbr_batch_size),
    verbose=1
)

# Question 6 : Sauvegarder le modèle final
??????

# 10. Tester le modèle (étape pouvant prendre quelques minutes)

## Charger une image de test depuis internet


In [0]:
!wget -O fire.jpg https://media.wired.com/photos/5be5baad89450468242a14ba/master/pass/CampFire-1059476842.jpg

## Tester modèle avec l'image chargée "fire.jpg"

In [0]:
%matplotlib inline

model_path="results/VGG16/classification_final.h5"   # Ici, vous devez indiquer votre modèle
classes_path = "classes.txt"
image_path="fire.jpg"                # Votre image de test   
top_n=3
model = load_model(model_path)

# load class names
classes = []
with open(classes_path, 'r') as f:
    classes = list(map(lambda x: x.strip(), f.readlines()))

img = image.load_img(image_path, target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# predict
pred = model.predict(x)[0]
result = [(classes[i], float(pred[i]) * 100.0) for i in range(len(pred))]
result.sort(reverse=True, key=lambda x: x[1])

img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
font = cv2.FONT_HERSHEY_COMPLEX 

for i in range(top_n):
    (class_name, prob) = result[i]
    textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
    textX = (img.shape[1] - textsize[0]) / 2
    textY = (img.shape[0] + textsize[1]) / 2
    if (i == 0) :
        cv2.putText(img, class_name, (int(textX)-100, int(textY)), font, 5, (255,255,255), 6, cv2.LINE_AA)
    print("Top %d ====================" % (i + 1))
    print("Class name: %s" % (class_name))
    print("Probability: %.2f%%" % (prob))
plt.imshow(img)
plt.show()